This program use the **Tweepy** library to get the tweets from Twitter API using Python and passes them to the Spark Streaming instance.
$\newline$First, we set up our **Twitter credentials** to have the authorization to take tweets from it. Next, we will create a TweetsListener, which is a class that inherit from the StreamLister class, and it would be responsible for the streaming itself. For initialization, we provide a socket object and implement two methods : on_data and on_error. The first method is responsible for receiving data from the Twitter stream and sending it to a socket. The second is used for receiving error messages and printing it.
$\newline$ Then, we connect to Twitter streaming by using the **sendTweets** method which will have a socket object as a parameter and create a stream object instance.
$\newline$Finally, to start streaming, we create a socket object, obtain the local machine name which is **'127.0.0.1'**, reserve a port for your service on the host, bind to the port, wait for a client connection which would be from the **streaming** notebook and call **sendTweets** method to build a new connection, and here we will pick all the tweets containing the covid and newyear tag.

In [ ]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

# Set up the Twitter credentials
access_token = ''
access_secret = ''
consumer_key = ''
consumer_secret = ''


# We will create a TweetsListener
class TweetsListener(StreamListener):

    def __init__(self, csocket):
        self.client_socket = csocket

    def on_data(self, data):
        try:
            msg = json.loads(data)
            #print(msg['text'].encode('utf-8') )
            self.client_socket.send( msg['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: ",str(e))
        return True
    def on_error(self, status):
        print(status)
        return True


def sendTweets(c_socket):
    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=['covid','newyear'])

    
if __name__ == "__main__":
    auth = OAuthHandler(consumer_key, consumer_secret) # Authorize streaming with our credentials
    auth.set_access_token(access_token, access_secret)
    s = socket.socket()                                # Create a socket object
    host = "127.0.0.1"                                 # Get local machine name
    port = 9995                                        # Reserve a port for your service.
    s.bind((host, port))                               # Bind to the port
    print("Listening ...")
    s.listen(5)                                        # Waiting for client connection.
    c, addr = s.accept()                               # Establish connection with client.
    print("Received request from: ",addr)
    sendTweets(c)                                      # Send data to the server